In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_pickle("drive/MyDrive/df_pseudo_1k_missing.pkl")[:20000]
bbbp = pd.read_csv("drive/MyDrive/GNN_BBBP_Property_Final.csv")[:20000]
n = 0
for i in range(len(df)):
  if len(df['cmpdname'][i]) < 20 and df['fullDescription'][i].split(' ')[0] == df['cmpdname'][i]:
    n += 1
    # print(df['cmpdname'][i], "--------", df['fullDescription'][i])
    df['fullDescription'][i] = f'<s> [INST] What is the description of {df["cmpdname"][i]}? [/INST]' + df['fullDescription'][i] + " " + bbbp['Permeability Description'][i] + '</s>'
  else:
    df = df.drop(i)


In [ ]:
!pip install rdkit-pypi
!pip install periodictable
import periodictable as pt
elements = pt.elements
from collections import Counter
import rdkit
def create_desc(smiles):
  try:
    x = rdkit.Chem.MolFromSmiles(smiles)
    desc = f'Get description of molecule with {x.GetNumAtoms()} atoms and {x.GetNumHeavyAtoms()} heavy atoms given '
    desc += f"smiles: {smiles}\n"
  except:
    print("Returning None")
    return None
  for i in range(len(x.GetAtoms())):
    atom = x.GetAtoms()[i]
    el = elements[atom.GetAtomicNum()]
    bonds = atom.GetBonds()
    desc += f"Atom {i+1}: {el}, valency {atom.GetExplicitValence()}. It forms "
    atom_bonds = []
    for bond in bonds:
      # print(dir(bond))
      bond_type = str(bond.GetBondType()).lower()
      atom_start_id = bond.GetBeginAtomIdx()
      atom_end_id = bond.GetEndAtomIdx()
      start_atom = x.GetAtoms()[atom_start_id]
      end_atom = x.GetAtoms()[atom_end_id]
      if start_atom == i:
        atom_bonds.append(f'{bond_type} bond with {elements[end_atom.GetAtomicNum()]} with valency contribution of {bond.GetValenceContrib(atom)}')
      else:
        atom_bonds.append(f'{bond_type} bond with {elements[start_atom.GetAtomicNum()]} with valency contribution of {bond.GetValenceContrib(atom)}')
    count = Counter(atom_bonds)
    temp = ''
    for el in count:
      k = el
      if count[el] > 1:
        k = k.replace("bond", "bonds")
      temp += f"{count[el]} {k} and "
    temp = "".join(temp.rsplit("and ", 1))


    desc += temp

    desc += '\n'
  return desc

In [ ]:
df['cmpdname'][138]

'Bretylium'

In [ ]:
bbbp.iloc[138]

Unnamed: 0                                                                138
Permeability                                                           0.0829
Permeability Description    This molecule has a very low blood-brain barri...
Name: 138, dtype: object

In [ ]:
print(create_desc("CC[N+](C)(C)CC1=CC=CC=C1Br"))

Get description of molecule with 13 atoms and 13 heavy atoms given smiles: CC[N+](C)(C)CC1=CC=CC=C1Br
Atom 1: C, valency 1. It forms 1 single bond with C with valency contribution of 1.0 
Atom 2: C, valency 2. It forms 2 single bonds with C with valency contribution of 1.0 
Atom 3: N, valency 4. It forms 1 single bond with C with valency contribution of 1.0 and 3 single bonds with N with valency contribution of 1.0 
Atom 4: C, valency 1. It forms 1 single bond with N with valency contribution of 1.0 
Atom 5: C, valency 1. It forms 1 single bond with N with valency contribution of 1.0 
Atom 6: C, valency 2. It forms 1 single bond with N with valency contribution of 1.0 and 1 single bond with C with valency contribution of 1.0 
Atom 7: C, valency 4. It forms 1 single bond with C with valency contribution of 1.0 and 2 aromatic bonds with C with valency contribution of 1.5 
Atom 8: C, valency 3. It forms 2 aromatic bonds with C with valency contribution of 1.5 
Atom 9: C, valency 3. It for

In [ ]:
# for i in range(len(df['fullDescription'])):
#   df['fullDescription'][i] = f'<s>[INST] {create_desc(df["isosmiles"][i])} [/INST] ' + bbbp['Permeability Description'][i] + '</s>'

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19

In [ ]:
from datasets import load_dataset
import datasets

# dataset = load_dataset('/drive/MyDrive/description_df.pkl', split="train")

In [ ]:
data = pd.DataFrame(df, columns=['fullDescription'])

In [ ]:
data=data.rename(columns={'fullDescription': 'text'})

In [ ]:
import datasets

dataset = datasets.Dataset.from_pandas(data)


In [ ]:
len(dataset)

5252

In [ ]:
del df, data

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

# model_name = "TinyPixel/Llama-2-7B-bf16-sharded"
model_name = "huggyllama/llama-13b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.2
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
from transformers import TrainingArguments

output_dir = "results"
per_device_train_batch_size = 3
gradient_accumulation_steps = 3
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = -1
warmup_ratio = 0.03
lr_scheduler_type = "constant"
num_train_epochs = 1

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)


In [ ]:
from trl import SFTTrainer

max_seq_length = 888

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/5252 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
dataset[534]

{'text': '<s> [INST] What is the description of Olomoucine? [/INST]Olomoucine is a 9H-purine that is substituted by a (2-hydroxyethyl)nitrilo, benzylnitrilo and a methyl group at positions 2,6 and 9, respectively. It is a cyclin-dependent kinase inhibitor. It has a role as an EC 2.7.11.22 (cyclin-dependent kinase) inhibitor. It is a member of 2,6-diaminopurines and a member of ethanolamines. The molecule weighs 298.34 g/mol. The compound consists of 15 carbon atoms, 18 hydrogen atoms, 6 nitrogen atoms, 1 oxygen atom and a molecular formula of C15H18N6O. The molecule exhibits a polar area of 87.90 square angstroms. The compound has 3 hydrogen bond donor(s). This molecule has a very high blood-brain barrier permeability on a 0-1 scale (0.97).</s>'}

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.934200
20,1.714900
30,1.496700
40,1.173300
50,0.984700
60,0.872700
70,0.814900
80,0.820800
90,0.729400
100,0.721100


Step,Training Loss
10,1.934200
20,1.714900
30,1.496700
40,1.173300
50,0.984700
60,0.872700
70,0.814900
80,0.820800
90,0.729400
100,0.721100


TrainOutput(global_step=583, training_loss=0.7209402596766503, metrics={'train_runtime': 11618.1935, 'train_samples_per_second': 0.452, 'train_steps_per_second': 0.05, 'total_flos': 9.281703114768384e+16, 'train_loss': 0.7209402596766503, 'epoch': 1.0})

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs4")
model_to_save.save_pretrained("drive/MyDrive/outputs4")

lora_config = LoraConfig.from_pretrained('outputs4')
model = get_peft_model(model, lora_config)

In [ ]:
dataset[4]

{'text': '<s> [INST] What is the description of Dihydrogenphosphate? [/INST]Dihydrogenphosphate is a monovalent inorganic anion that consists of phosphoric acid in which one of the three OH groups has been deprotonated. It is a monovalent inorganic anion and a phosphate ion. It is a conjugate base of a phosphoric acid. It is a conjugate acid of a hydrogenphosphate., See also: Phosphoric Acid (conjugate). It has a molecular weight of 96.99 g/mol. The molecule has the molecular formula of 2 hydrogen atoms, 4 oxygen atoms, 1 phosphorus atom and a molecular formula of H2O4P- with a total charge of negative -1. It has a polar area of 80.60 square angstroms. The molecule possesses 2 hydrogen bond donor(s). This molecule has a very low blood-brain barrier permeability on a 0-1 scale (0.02).</s>'}

In [ ]:
text = f'<s> [INST] What is the description of 1,4-Dideoxy-1,4-iminohexitol? [/INST]'
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=888)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
!pip install huggingFace_hub
!pip install huggingface-cli

ERROR: Could not find a version that satisfies the requirement huggingface-cli (from versions: none)
ERROR: No matching distribution found for huggingface-cli


In [ ]:
!huggingface-cli login

model.push_to_hub("repo name")
tokenizer.push_to_hub("repo name")